In [2]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [172]:
import os
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import geopandas as gpd
from azure.storage.blob import ContainerClient
import matplotlib.pyplot as plt
import warnings
import plotly.express as px

# Raised as part of interpolating with Pandas
warnings.simplefilter(action="ignore", category=FutureWarning)

load_dotenv()

ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]
THRESHOLD_SPEED_OPT1 = 89  # severe topical storm
THRESHOLD_SPEED_OPT2 = 118  # typhoon

AA_DATA_DIR = Path(os.getenv("AA_DATA_DIR"))
AA_DATA_DIR_NEW = Path(os.getenv("AA_DATA_DIR_NEW"))

DEV_BLOB_SAS = os.getenv("DSCI_AZ_SAS_DEV")
DEV_BLOB_NAME = "imb0chd0dev"
DEV_BLOB_URL = f"https://{DEV_BLOB_NAME}.blob.core.windows.net/"
DEV_BLOB_PROJ_URL = DEV_BLOB_URL + "projects" + "?" + DEV_BLOB_SAS
GLOBAL_CONTAINER_NAME = "global"
DEV_BLOB_GLB_URL = DEV_BLOB_URL + GLOBAL_CONTAINER_NAME + "?" + DEV_BLOB_SAS

dev_glb_container_client = ContainerClient.from_container_url(DEV_BLOB_GLB_URL)

## 1. Loading in raw data

In [4]:
emdat_path = (
    AA_DATA_DIR_NEW
    / "private"
    / "processed"
    / "glb"
    / "emdat"
    / "emdat-tropicalcyclone-2000-2022-processed-sids.csv"
)

ibtracs_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "glb"
    / "ibtracs"
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)

adm1_path = (
    AA_DATA_DIR
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)

In [138]:
df_emdat = pd.read_csv(emdat_path)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_ibtracs = gpd.read_file(ibtracs_path)

gdf_adm1_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]
df_emdat = df_emdat[df_emdat.iso3 == "MOZ"]  # Just want the impacts to Mozambique

Clean up EM-DAT data to get list of impactful cyclones that hit landfall in our AOI in Mozambique. Combine with IBTrACS data.

In [6]:
df_emdat_sel = df_emdat[df_emdat.iso3 == "MOZ"]

gdf_hist = gdf_ibtracs.merge(df_emdat_sel, left_on="SID", right_on="sid", how="right")

gdf_hist = gdf_hist[
    ["SID", "NAME", "Total Affected", "ISO_TIME", "REU_WIND", "geometry"]
].sort_values(by="Total Affected")

df_impact_summary = (
    gdf_hist.groupby(["SID", "NAME"]).max("Total Affected").reset_index()
)

In [7]:
# TODO: Split up by province
df_impact_summary

SID         NAME  Total Affected  REU_WIND
0   2000032S11116  ELINE:LEONE             NaN     100.0
1   2000083S17102        HUDAH           304.0     120.0
2   2003056S21042       JAPHET         23010.0      95.0
3   2007043S11071        FAVIO        162770.0     105.0
4   2008062S10064        JOKWE        220013.0     105.0
5   2009082S16039       IZILDA          7103.0      60.0
6   2012010S24049        DANDO         40042.0      45.0
7   2012018S16041        FUNSO         65000.0     110.0
8   2012056S13057        IRINA          4958.0      52.0
9   2017043S19040        DINEO        750102.0      75.0
10  2019063S18038         IDAI       1501500.0     105.0
11  2019112S10053      KENNETH        400094.0     115.0
12  2021012S12086       ELOISE        481901.0      80.0

In [139]:
# combining US Wind and REUnion for those with no wind speed
gdf_ibtracs["REU_USA_WIND"] = gdf_ibtracs["REU_WIND"].fillna(
    gdf_ibtracs["USA_WIND"] * 0.88
)

df_ibtracs = gdf_ibtracs[["SID", "NAME", "REU_USA_WIND", "ISO_TIME", "LAT", "LON"]]
df_ibtracs.loc[:, "ISO_TIME"] = pd.to_datetime(df_ibtracs["ISO_TIME"])
df_ibtracs = df_ibtracs.dropna(subset="REU_USA_WIND")

# Interpolate the IBTrACS points to 30min frequency
gdfs = []
for sid, group in df_ibtracs.groupby("SID"):
    df_interp = (
        group.set_index("ISO_TIME").resample("30min").interpolate().reset_index()
    )
    df_interp["SID"] = sid
    df_interp["NAME"] = df_interp.iloc[0].NAME
    gdf_interp = gpd.GeoDataFrame(
        data=df_interp,
        geometry=gpd.points_from_xy(df_interp["LON"], df_interp["LAT"]),
        crs=4326,
    )
    gdfs.append(gdf_interp)

gdf_ibtracs_interp = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

In [140]:
# Filter out the points that don't make landfall with the coastal provinces in Mozambique
gdf_ibtracs_interp_sel = gpd.sjoin(
    gdf_ibtracs_interp, gdf_adm1_sel, how="inner", predicate="intersects"
)[
    [
        "ISO_TIME",
        "SID",
        "NAME",
        "REU_USA_WIND",
        "LAT",
        "LON",
        "geometry",
        "ADM1_PT",
        "ADM1_PCODE",
    ]
]

In [162]:
# Now group and join to get one row per event, with relevant variables added in

df_ibtracs_sum = (
    gdf_ibtracs_interp_sel.groupby(["SID", "NAME"])
    .agg({"REU_USA_WIND": "max", "ISO_TIME": "min"})
    .reset_index()
)

# TODO: Why are there some NA ADM1 values?
df_cyclone_summary = (
    df_ibtracs_sum.merge(
        gdf_ibtracs_interp_sel[["SID", "NAME", "ADM1_PT", "REU_USA_WIND", "ISO_TIME"]],
        on=["SID", "NAME", "REU_USA_WIND", "ISO_TIME"],
        how="left",
    )
    .merge(
        df_emdat[["sid", "Total Affected"]], left_on="SID", right_on="sid", how="left"
    )
    .drop(axis=1, columns=["sid"])
    .rename(
        columns={
            "REU_USA_WIND": "MAX_SPEED_ON_LAND",
            "ADM1_PT": "ADM1_OF_MAX",
            "ISO_TIME": "TIME_OF_MAX",
            "Total Affected": "TOTAL_AFFECTED",
        }
    )
)

# Note that these classifications are based on JUST speeds while the cyclone is over our AOI
# So may not necessarily reflect the global classification of the cyclone
df_cyclone_summary["SEVERE_TROPICAL_STORM"] = (
    df["MAX_SPEED_ON_LAND"] >= THRESHOLD_SPEED_OPT1
) & (df["MAX_SPEED_ON_LAND"] < THRESHOLD_SPEED_OPT2)
df_cyclone_summary["TYPHOON"] = df["MAX_SPEED_ON_LAND"] >= THRESHOLD_SPEED_OPT2

# Also create a column with just the day, for matching against daily precipitation data
df_cyclone_summary["DAY_OF_MAX"] = pd.to_datetime(
    df_cyclone_summary["TIME_OF_MAX"].dt.strftime("%Y-%m-%d")
)

In [163]:
df_cyclone_summary

SID         NAME  MAX_SPEED_ON_LAND         TIME_OF_MAX  \
0   1978347S20041       ANGELE          72.000000 1978-12-19 01:00:00   
1   1981351S12060    BENEDICTE          58.208333 1981-12-24 03:30:00   
2   1982030S15065      ELECTRE          26.400000 1982-02-05 18:00:00   
3   1984016S15073      DOMOINA          44.000000 1984-01-28 08:00:00   
4   1986005S15043      BEROBIA          42.000000 1986-01-09 19:30:00   
5   1987325S06050      UNNAMED          22.000000 1987-11-25 22:00:00   
6   1988055S10051        FILAO          51.000000 1988-03-01 21:30:00   
7   1994076S14078        NADIA          66.440000 1994-03-24 17:30:00   
8   1995020S20038        FODAH          13.200000 1995-01-21 00:00:00   
9   1996001S08075       BONITA          64.240000 1996-01-13 20:30:00   
10  1997056S15041      LISETTE          59.840000 1997-02-25 16:00:00   
11  1998016S12043      UNNAMED          30.800000 1998-01-17 05:30:00   
12  1998032S18036      BELTANE          22.000000 1998-02-01 00:00:00   
13  1999357S08076      ASTRIDE          30.666667 2000-01-03 05:00:00   
14  2000032S11116  ELINE:LEONE          94.000000 2000-02-22 07:30:00   
15  2000083S17102        HUDAH          66.000000 2000-04-08 06:00:00   
16  2002364S16045      DELFINA          47.500000 2002-12-31 21:30:00   
17  2003056S21042       JAPHET          71.500000 2003-03-02 16:30:00   
18  2007043S11071        FAVIO          92.000000 2007-02-22 10:00:00   
19  2008062S10064        JOKWE          80.833333 2008-03-08 14:30:00   
20  2009082S16039       IZILDA          20.000000 2009-03-22 12:00:00   
21  2012010S24049        DANDO          43.000000 2012-01-16 09:00:00   
22  2013046S20042       HARUNA          20.000000 2013-02-16 12:00:00   
23  2014048S16039        GUITO          23.000000 2014-02-17 00:00:00   
24  2014086S10041       HELLEN          15.000000 2014-04-04 15:30:00   
25  2015013S18038       CHEDZA          21.333333 2015-01-13 06:00:00   
26  2017043S19040        DINEO          75.000000 2017-02-15 17:30:00   
27  2019018S24033      DESMOND          35.000000 2019-01-17 15:30:00   
28  2019063S18038         IDAI          85.000000 2019-03-04 12:30:00   
29  2019112S10053      KENNETH          20.000000 2019-04-27 00:30:00   
30  2020355S11065      CHALANE          57.500000 2020-12-30 01:30:00   
31  2021012S12086       ELOISE          76.666667 2021-01-23 02:00:00   
32  2021042S23040       GUAMBE          28.333333 2021-02-12 02:30:00   
33  2022020S13059          ANA          42.826667 2022-01-24 08:00:00   
34  2022065S16055        GOMBE          86.533333 2022-03-11 00:30:00   
35  2022110S12051      JASMINE          30.800000 2022-04-23 05:30:00   
36  2023061S22036       FREDDY          78.613333 2023-03-11 17:30:00   
37  2024071S20039       FILIPO          45.466667 2024-03-12 05:00:00   

   ADM1_OF_MAX  TOTAL_AFFECTED  SEVERE_TROPICAL_STORM  TYPHOON DAY_OF_MAX  
0      Nampula             NaN                  False    False 1978-12-19  
1     Zambezia             NaN                  False    False 1981-12-24  
2     Zambezia             NaN                  False    False 1982-02-05  
3    Inhambane             NaN                  False    False 1984-01-28  
4       Sofala             NaN                  False    False 1986-01-09  
5      Nampula             NaN                  False    False 1987-11-25  
6     Zambezia             NaN                  False    False 1988-03-01  
7      Nampula             NaN                  False    False 1994-03-24  
8     Zambezia             NaN                  False    False 1995-01-21  
9          NaN             NaN                  False    False 1996-01-13  
10         NaN             NaN                  False    False 1997-02-25  
11         NaN             NaN                  False    False 1998-01-17  
12      Sofala             NaN                  False    False 1998-02-01  
13     Nampula             NaN                  False    False 2000-01-03  
14      Sofala            

Now load in the precipitation data from the processed IMERG files

In [156]:
imerg_path = (
    Path(AA_DATA_DIR) / "public" / "processed" / "moz" / "daily_imerg_precip_adm1.csv"
)

# TODO: Take the 2-day rolling sum?
df_imerg = pd.read_csv(imerg_path)
df_imerg = df_imerg[df_imerg.ADM1.isin(ADMS)]
df_imerg["date"] = pd.to_datetime(df_imerg["date"])

Create a simple plot to sanity check

In [225]:
fig = px.line(
    df_imerg_sel,
    x="date",
    y="total_precipitation",
    facet_row="ADM1",
    title="IMERG Observed Daily Precipitation Across Selected Provinces in Mozambique",
    template="simple_white",
)
fig.update_yaxes(tickangle=45)
fig.show()

In [193]:
df_cyclone_precip = df_cyclone_summary.merge(
    df_imerg, left_on="DAY_OF_MAX", right_on="date", how="left"
)

df_cyclone_precip = (
    df_cyclone_precip.drop(columns=["Unnamed: 0", "date"])
    .rename(columns={"total_precipitation": "TOTAL_PRECIPITATION"})
    .dropna(subset="TOTAL_PRECIPITATION") # Since we don't have precipitation before 2003
)

For cyclone events, what trends are there in the observed daily total precipitation per province and the maximum windspeed over land in Mozambique? We can break this down by province, however the max wind speed is independent of province and is the max across the total overall area. We can also color points by the estimates of impact (total affected population in the country) for each cyclone. Note that many of the cyclones from IBTrACS don't have EM-DAT estimates.

Some observations:
- We do see that higher impact cyclones **do** often have higher max wind speeds (Idai and Dineo, for example)
- Observed daily precipitation is not necessarily a good indicator of the cyclone hitting. Cyclone ANA had a 

Next steps:
- Need to do some more investigation on approaches for aggregating or summing up precipitation from IMERG. Could be: calculating 2 or 3 day rolling sums
- Might also be worth calculating return periods for precipitation values to identify what might be notable outlier observations

In [221]:
fig = px.scatter(
    df_cyclone_precip,
    x="TOTAL_PRECIPITATION",
    y="MAX_SPEED_ON_LAND",
    color="TOTAL_AFFECTED",
    facet_col="ADM1",
    template="simple_white",
    color_continuous_scale=px.colors.sequential.Reds[2:7],
    hover_data=["NAME", "DAY_OF_MAX"],
    title="Daily Precipitation vs Max Wind Speed for Cyclones Making Landfall in Mozambique",
)
fig.update_traces(marker_size=10)
fig.show()